In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import pandas_data_util
import tensorflow as tf

In [7]:
filename = 'data/bq-results-20211105-102236-1pg8as4q8dgo.csv'
train_percent=0.6
valid_percent=0.2
normalize=0
horizon=0
window=5
Data = pandas_data_util.DataUtil(filename,
                train_percent,
                valid_percent,
                horizon,
                window,
                normalize)

/home/derek/projects/ml/tf_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3361: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,14,20,45,58,59,82,83,84) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [8]:
Data.train[0]

array([[[Timestamp('2021-08-03 23:30:18.996356+0000', tz='UTC'),
         0.08252671512388611, -0.7730977866761316, ..., 1.0, 0.0, 'bbr'],
        [Timestamp('2021-08-04 19:42:12.895446+0000', tz='UTC'),
         0.03454523416826806, -0.7716885806849644, ..., 1.0, 0.0, 'bbr'],
        [Timestamp('2021-08-05 20:28:10.105184+0000', tz='UTC'),
         -0.04741356344491683, -0.7673525622506039, ..., 1.0, 0.0,
         'bbr'],
        [Timestamp('2021-08-06 21:28:02.679139+0000', tz='UTC'),
         0.0685225508258349, -0.7721221825284005, ..., 1.0, 0.0, 'bbr'],
        [Timestamp('2021-08-07 22:42:00.336785+0000', tz='UTC'),
         0.07632815059851918, -0.7713633793023874, ..., 1.0, 0.0, 'bbr']],

       [[Timestamp('2021-08-04 19:42:12.895446+0000', tz='UTC'),
         0.03454523416826806, -0.7716885806849644, ..., 1.0, 0.0, 'bbr'],
        [Timestamp('2021-08-05 20:28:10.105184+0000', tz='UTC'),
         -0.04741356344491683, -0.7673525622506039, ..., 1.0, 0.0,
         'bbr'],
      